In [19]:
# Dependencies and Setup
import pandas as pd
import csv
import os
import numpy as np

## Load and Clean Data

In [20]:
# Files to Load
cont_code_path = os.path.join(os.getcwd(), "Resources", "country-and-continent-codes-list-cleaned.csv")
teams_path = os.path.join(os.getcwd(), "Resources", "highest_earning_teams.csv")
players_path = os.path.join(os.getcwd(), "Resources", "highest_earning_players.csv")
time_trends_path = os.path.join(os.getcwd(), "Resources", "time_trends.csv")
trends_path = os.path.join(os.getcwd(), "Resources", "trend_map.csv")
general_data_path = os.path.join(os.getcwd(), "Resources", "GeneralEsportData.csv")
historical_data_path = os.path.join(os.getcwd(), "Resources", "HistoricalEsportData.csv")

In [21]:
# Read Files and Store Into Pandas Data Frame
cont_code_df = pd.read_csv(cont_code_path)
teams_df = pd.read_csv(teams_path)
players_df = pd.read_csv(players_path)
time_trends_df = pd.read_csv(time_trends_path)
trends_geo_df = pd.read_csv(trends_path)
general_data_df = pd.read_csv(general_data_path, encoding='cp1252')
historical_data_df = pd.read_csv(historical_data_path, encoding='cp1252')

In [22]:
## Look at our data
cont_code_df.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,Afghanistan,AF,AFG,4.0
1,Europe,EU,Albania,AL,ALB,8.0
2,Antarctica,AN,Antarctica,AQ,ATA,10.0
3,Africa,AF,Algeria,DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [23]:
teams_df.head()

,TeamId,TeamName,TotalUSDPrize,TotalTournaments,Game,Genre
0,760,San Francisco Shock,3105000.0,7,Overwatch,First-Person Shooter
1,776,London Spitfire,1591136.5,13,Overwatch,First-Person Shooter
2,768,New York Excelsior,1572618.5,18,Overwatch,First-Person Shooter
3,773,Philadelphia Fusion,1186278.5,15,Overwatch,First-Person Shooter
4,766,Seoul Dynasty,1130000.0,6,Overwatch,First-Person Shooter


In [24]:
players_df.head()

,PlayerId,NameFirst,NameLast,CurrentHandle,CountryCode,TotalUSDPrize,Game,Genre
0,3883,Peter,Rasmussen,dupreeh,dk,1822989.41,Counter-Strike: Global Offensive,First-Person Shooter
1,3679,Andreas,Højsleth,Xyp9x,dk,1799288.57,Counter-Strike: Global Offensive,First-Person Shooter
2,3885,Nicolai,Reedtz,dev1ce,dk,1787489.88,Counter-Strike: Global Offensive,First-Person Shooter
3,3672,Lukas,Rossander,gla1ve,dk,1652350.75,Counter-Strike: Global Offensive,First-Person Shooter
4,17800,Emil,Reif,Magisk,dk,1416448.64,Counter-Strike: Global Offensive,First-Person Shooter


In [25]:
time_trends_df = time_trends_df.drop([0])
## Need to fix headers
new_header = time_trends_df.iloc[0]
time_trends_df = time_trends_df[1:]
time_trends_df.columns = new_header
time_trends_df = time_trends_df.rename(columns={"League of Legends: (Worldwide)" : "League_of_Legends", 
                                              "Dota 2: (Worldwide)" : "Dota_2",
                                              "Fortnite: (Worldwide)" : "Fortnite",
                                              "PlayerUnknown's Battlegrounds: (Worldwide)" : "PUBG",
                                              "Counter-Strike: Global Offensive: (Worldwide)" : "CSGO"
                                             })
time_trends_df.head()

1,Week,League_of_Legends,Dota_2,Fortnite,PUBG,CSGO
2,9/6/2020,26,5,74,76,11
3,9/13/2020,26,5,72,53,11
4,9/20/2020,27,5,72,45,11
5,9/27/2020,31,4,65,41,11
6,10/4/2020,30,4,57,39,10


In [26]:
trends_geo_df = trends_geo_df.drop([0])
## Need to fix headers
new_header = trends_geo_df.iloc[0]
trends_geo_df = trends_geo_df[1:]
trends_geo_df.columns = new_header
trends_geo_df = trends_geo_df.rename(columns={"League of Legends: (9/1/20 - 9/1/21)" : "League_of_Legends", 
                                              "Dota 2: (9/1/20 - 9/1/21)" : "Dota_2",
                                              "Fortnite: (9/1/20 - 9/1/21)" : "Fortnite",
                                              "PlayerUnknown's Battlegrounds: (9/1/20 - 9/1/21)" : "PUBG",
                                              "Counter-Strike: Global Offensive: (9/1/20 - 9/1/21)" : "CSGO"
                                             })
trends_geo_df.head()

1,Albania,League_of_Legends,Dota_2,Fortnite,PUBG,CSGO
2,Algeria,<1%,<1%,9%,91%,<1%
3,American Samoa,2%,1%,3%,94%,<1%
4,Andorra,8%,11%,3%,75%,3%
5,Angola,NaN,NaN,NaN,100%,NaN
6,Anguilla,1%,<1%,7%,92%,<1%


In [27]:

general_data_df.drop(columns = ["Genre"]).head()

,Game,ReleaseDate,TotalEarnings,OnlineEarnings,TotalPlayers,TotalTournaments
0,Age of Empires,1997,190262.81,156839.89,255,96
1,Age of Empires II,1999,2008941.78,619582.87,1392,1031
2,Age of Empires III,2005,68613.85,41500.00,108,77
3,Age of Empires Online,2011,4853.56,775.00,20,13
4,Age of Mythology,2002,52360.00,52000.00,22,6


## Create Database

In [28]:
## Dependencies
from sqlalchemy import create_engine
from hiddenConfig import password
import psycopg2

In [30]:
## establishing the connection with database. This checks whether or not the database exists before creation. If it is,
## it will tell you. Then move onto the next cell
conn = None
try:
    conn = psycopg2.connect(
       database="postgres", user='postgres', password=password, host='127.0.0.1', port= '5432'
    )

except:
    print('Database not connected.')
    
if conn is not None:
    conn.autocommit = True

    #Creating a cursor object
    cursor = conn.cursor()

    cursor.execute("SELECT datname FROM pg_database;")
    
    list_database = cursor.fetchall()
    database_name = 'esports_db'
    
    if (database_name,) in list_database:
        print(f"'{database_name}' Database already exists")
    else:
        #Preparing query to create a database
        sql = '''CREATE database esports_db''';

        #Creating a database
        cursor.execute(sql)
        print("Database created successfully!")

#Closing the connection
conn.close()

Database not connected.


AttributeError: 'NoneType' object has no attribute 'close'

In [31]:
## Connect to local database
rds_connection_string = f"postgres:{password}@localhost:5432/esports_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [32]:
## Check to see if there are any tables already in the database
engine.table_names()

C:\Users\1800h\AppData\Local\Temp/ipykernel_27540/2263100480.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/14/e3q8)

#### Load the dataframes into our database

In [ ]:
cont_code_df.to_sql(name='cont_code_data', con=engine, if_exists='replace', index=False)
teams_df.to_sql(name='team_data', con=engine, if_exists='replace', index=False)
trends_geo_df.to_sql(name='trends_geo_data', con=engine, if_exists='replace', index=False)
players_df.to_sql(name='player_data', con=engine, if_exists='replace', index=False)
time_trends_df.to_sql(name='time_trends_data', con=engine, if_exists='replace', index=False)
general_data_df.to_sql(name='general_data', con=engine, if_exists='replace', index=False)
historical_data_df.to_sql(name='historical_data', con=engine, if_exists='replace', index=False)

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/14/e3q8)

#### Check to make sure it worked OK

In [ ]:
engine.execute('select * from cont_code_data').first()

In [ ]:
engine.execute('select * from team_data').first()

In [ ]:
engine.execute('select * from trends_geo_data').first()

In [ ]:
engine.execute('select * from player_data').first()

In [ ]:
engine.execute('select * from time_trends_data').first()